In [1]:
from bs4 import BeautifulSoup
import requests
from selenium import webdriver



In [156]:
driver=webdriver.Chrome()

driver.get('https://www.youtube.com/@HealthyGamerGG/videos')



In [66]:
soup=BeautifulSoup(driver.page_source,'html.parser')

In [67]:
data=[]
sp=soup.find('div',id='contents')
for sp in sp.find_all('ytd-rich-item-renderer',class_='style-scope ytd-rich-grid-renderer'):
    title=sp.find('a',id='video-title-link').get('title')
    link='https://www.youtube.com'+sp.find('a').get('href')
    # views=sp.find_all('span',class_='inline-metadata-item style-scope ytd-video-meta-block')[0].text
    # date=sp.find_all('span',class_='inline-metadata-item style-scope ytd-video-meta-block')[1].text
    img=sp.find('img').get('src').split('?')[0]
    duration=sp.find('div',class_='badge-shape-wiz__text').text
    # data.append({'title':title,'link':link,'views':views,'uploaded':date,'img':img,'duration':duration})
    data.append({'title':title,'link':link,'img':img,'duration':duration})
data    

[{'title': 'How Loneliness Makes Men More Addicted',
  'link': 'https://www.youtube.com/watch?v=uBiCK84EW38',
  'views': '161K views',
  'uploaded': '5 days ago',
  'img': 'https://i.ytimg.com/vi/uBiCK84EW38/hqdefault.jpg',
  'duration': '34:15'},
 {'title': 'Why Your Thoughts Aren’t The Enemy',
  'link': 'https://www.youtube.com/watch?v=TSBd_Ku1Zf0',
  'views': '189K views',
  'uploaded': '11 days ago',
  'img': 'https://i.ytimg.com/vi/TSBd_Ku1Zf0/hqdefault.jpg',
  'duration': '29:58'},
 {'title': 'Why Brainwashing Yourself Keeps You Stuck in Life',
  'link': 'https://www.youtube.com/watch?v=oFcQcmZJQ_k',
  'views': '195K views',
  'uploaded': '2 weeks ago',
  'img': 'https://i.ytimg.com/vi/oFcQcmZJQ_k/hqdefault.jpg',
  'duration': '29:20'},
 {'title': 'Why You Struggle to Follow Through (Thinkers vs Doers)',
  'link': 'https://www.youtube.com/watch?v=xSBGYoS6z68',
  'views': '445K views',
  'uploaded': '3 weeks ago',
  'img': 'https://i.ytimg.com/vi/xSBGYoS6z68/hqdefault.jpg',
  'dur

In [69]:
import pandas as pd
df=pd.DataFrame(data)

df.isnull().sum()

# df.to_csv('HealthyGamerGG-scraped_data.csv',index=False)

In [208]:
import time
from tqdm import tqdm
import re
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

data1=[]
for i,link in tqdm(enumerate(df['link'])):
    if i<=len(data1):
        continue

    driver.get(link)
    time.sleep(1)

    driver.execute_script("window.scrollTo(0, 700);")
    time.sleep(5)

    # Click the "Show More" button to expand the description
    more_button = driver.find_element(By.XPATH, '//tp-yt-paper-button[@id="expand"]')
    # driver.execute_script("arguments[0].scrollIntoView(true);", more_button)
    more_button.click()
    time.sleep(1)  # Wait for the description to expand
    

    soup=BeautifulSoup(driver.page_source,'html.parser')
    views=re.sub(r'[^\d]','',soup.find_all('span',class_='style-scope yt-formatted-string bold')[0].text)
    upload_date=soup.find_all('span',class_='style-scope yt-formatted-string bold')[2].text
    likes=soup.find_all('div',class_='yt-spec-button-shape-next__button-text-content')[5].text
    comments=re.sub(r'[^\d]','',soup.find('yt-formatted-string',class_='count-text style-scope ytd-comments-header-renderer').text)
    
    description=''
    
    disclaimer=soup.find('span',class_='yt-core-attributed-string yt-core-attributed-string--white-space-pre-wrap')
    for x in disclaimer.find_all('span',class_='yt-core-attributed-string--link-inherit-color'):
        description+=re.sub(r'<[^>]*>','',str(x)) + ' '
    
    data1.append({'views':views,'upload_date':upload_date,'likes':likes,'comments':comments,'description':description})
    driver.quit()


807it [06:56,  1.94it/s]


In [209]:

df1=pd.DataFrame(data1)
if df1.isnull().sum().sum()==0:
    print('No missing values')
    merged_df=pd.concat([df,df1],axis=1)
    merged_df.to_csv('HealthyGamerGG-scraped_data.csv',index=False)
else:
    print('Missing values found')
    print(df1.isnull().sum())
    print('Data not saved')

No missing values
